In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
import open3d as o3d
from estimate_object_pose import BoxSegmentation, BoxPoseEstimation
from realsense_camera import RealsenseCamera
from camera_utils import stream
from copy import deepcopy
from time import time, sleep
from aruco import detect_aruco

def show(image):
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# initialization

camera = RealsenseCamera(enable_depth=True)

K = camera.K
D = camera.D
depth_scale = camera.depth_scale

# box = "white"
box = "green"
# box = "red"
assert box in ("white", "green", "red")

if box == "white":
    edges_sizes = np.array([0.11, 0.05, 0.022])
if box == "green":
    edges_sizes = np.array([0.159, 0.079, 0.030])
if box == "red":
    edges_sizes = np.array([0.079, 0.079, 0.079])

box_segmentation = BoxSegmentation(erosion_size=0)
box_pose_estimation = BoxPoseEstimation(
    edges_sizes=edges_sizes, edge_points_per_cm=7,
    voxel_size=0.005, depth_scale=depth_scale, K=K, D=D,
    global_max_correspondence_distance=0.04,
    max_correspondence_distances=[0.04, 0.029, 0.018, 0.007])

In [ ]:
# test box segmentation

camera.start()
stream(camera)  # waits for a key to get image
image = camera.read()
camera.stop()

mask = box_segmentation.segment_box(image, box)
masked_image = image.copy()
masked_image[mask == 0] = 0
show(masked_image)

In [ ]:
# estimate box pose

camera.start()
stream(camera)  # waits for a key to get image
image, depth = camera.read(read_color=True, read_depth=True)
camera.stop()

mask = box_segmentation.segment_box(image, box)
box_pose = box_pose_estimation.estimate_pose(mask, depth)
if box_pose is None:
    print(f"Could not estimate pose. Reason: {box_pose_estimation.reason}\n")

In [ ]:
# visualize point clouds

transformed_box_pc = deepcopy(box_pose_estimation.gt_pc)
transformed_box_pc.transform(box_pose)
transformed_box_pc.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc = box_pose_estimation.extracted_pc
extracted_box_pc.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc, extracted_box_pc])

In [ ]:
# visualize point clouds down

transformed_box_pc_down = deepcopy(box_pose_estimation.gt_pc_down)
transformed_box_pc_down.transform(box_pose)
transformed_box_pc_down.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc_down = box_pose_estimation.extracted_pc_down
extracted_box_pc_down.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc_down, extracted_box_pc_down])

In [ ]:
# visualize box frame

force_z_axis_up = False
if force_z_axis_up:
    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        pose = np.matmul(box_pose, correction)
else:
    pose = box_pose

draw = image.copy()
rvec, _ = cv2.Rodrigues(pose[0:3, 0:3])
tvec = box_pose[0:3, 3]
cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.07)
show(draw)

In [ ]:
# stream depth

def draw_depth(key, image, depth):
    depth_cm = (depth * depth_scale * 100).clip(0, 255).astype(np.uint8)
    image[...] = np.tile(depth_cm, (3, 1, 1)).transpose(1, 2, 0)

camera.start()
stream(camera, callbacks=draw_depth)
camera.stop()

In [ ]:
# stream box segmentation

def segment_box(key, image, **kwargs):
    mask = box_segmentation.segment_box(image, box)
    image[mask == 0] //= 5

camera.start()
stream(camera, callbacks=segment_box)
camera.stop()

In [ ]:
# stream box frame

times = list()

def draw_box_frame(key, image, depth):
    tic = time()
    mask = box_segmentation.segment_box(image, box)
    box_pose = box_pose_estimation.estimate_pose(mask, depth)
    toc = time()

    if box_pose is None:
        return

    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        box_pose = np.matmul(box_pose, correction)

    rvec, _ = cv2.Rodrigues(box_pose[0:3, 0:3])
    tvec = box_pose[0:3, 3]
    cv2.drawFrameAxes(image, K, D, rvec, tvec, 0.07)

    times.append(toc - tic)

camera.start()
stream(camera, callbacks=draw_box_frame)
camera.stop()

print(f"{(sum(times) / len(times) * 1000):.1f} ms")

debug section

In [ ]:
# debug box frame

max_n = 100
images = list()
depths = list()
draws = list()

times = list()

def draw_box_frame(key, image, depth):
    images.append(image.copy())
    depths.append(depth.copy())

    tic = time()
    mask = box_segmentation.segment_box(image, box)
    box_pose = box_pose_estimation.estimate_pose(mask, depth)
    toc = time()

    if box_pose is None:
        draws.append(image.copy())
        if len(images) > max_n:
            images.pop(0)
            depths.pop(0)
            draws.pop(0)
        return

    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        box_pose = np.matmul(box_pose, correction)

    rvec, _ = cv2.Rodrigues(box_pose[0:3, 0:3])
    tvec = box_pose[0:3, 3]
    cv2.drawFrameAxes(image, K, D, rvec, tvec, 0.07)

    times.append(toc - tic)

    draws.append(image.copy())

    if len(images) > max_n:
        images.pop(0)
        depths.pop(0)
        draws.pop(0)

camera.start()
stream(camera, callbacks=draw_box_frame)
camera.stop()

print(f"{(sum(times) / len(times) * 1000):.1f} ms")

In [ ]:
for i, (image, depth, draw) in enumerate(zip(images, depths, draws)):
    print(i)
    show(draw)

In [ ]:
i = 0
show(draws[i])
image = images[i]
depth = depths[i]

In [ ]:
# stream point clouds

vis = o3d.visualization.Visualizer()
vis.create_window()

extracted_pc = o3d.geometry.PointCloud()
vis.add_geometry(extracted_pc)

gt_pc = deepcopy(box_pose_estimation.gt_pc)
gt_pc.paint_uniform_color(np.array([1, 0, 0]))
vis.add_geometry(gt_pc)

n_accum = 5
box_poses = list()
rvecs = list()
tvecs = list()

def show_point_cloud(key, image, depth):
    mask = box_segmentation.segment_box(image, box)
    box_pose = box_pose_estimation.estimate_pose(mask, depth)
    if box_pose is None:
        extracted_pc.points.clear()
        gt_pc.points.clear()
    else:
        if len(box_poses) > 0:
            box_pose = BoxPoseEstimation.align_box_poses(box_poses[-1], box_pose)
        rvec, _ = cv2.Rodrigues(box_pose[:3, :3])
        tvec = box_pose[:3, 3]
        box_poses.append(box_pose)
        rvecs.append(rvec)
        tvecs.append(tvec)
        if len(box_poses) > n_accum:
            box_poses.pop(0)
            rvecs.pop(0)
            tvecs.pop(0)

        rvec = sum(rvecs) / len(rvecs)
        tvec = sum(tvecs) / len(tvecs)
        box_pose = np.eye(4)
        box_pose[:3, :3] = cv2.Rodrigues(rvec)[0]
        box_pose[:3, 3] = tvec

        extracted_pc.points = deepcopy(box_pose_estimation.extracted_pc.points)
        extracted_pc.paint_uniform_color(np.array([0, 1, 0]))
        gt_pc.points = deepcopy(box_pose_estimation.gt_pc.points)
        gt_pc.paint_uniform_color(np.array([1, 0, 0]))
        gt_pc.transform(box_pose)
        # extracted_pc.transform(np.linalg.inv(box_pose))

    vis.update_geometry(extracted_pc)
    vis.update_geometry(gt_pc)
    vis.poll_events()
    vis.update_renderer()

camera.start()
stream(camera, callbacks=show_point_cloud)
camera.stop()

vis.destroy_window()

In [ ]:
# stream comparison of aruco and semantic box detection

assert box == "white"

rot_errors = list()
trans_errors = list()
last_box_pose = None

def callback(key, image, depth, **kwargs):
    mask = box_segmentation.segment_box(image, box)
    box_pose = box_pose_estimation.estimate_pose(mask, depth)
    if box_pose is None:
        return
    if last_box_pose is not None:
        box_pose = BoxPoseEstimation.align_box_poses(last_box_pose, box_pose)
    last_box_pose = box_pose

    arucos = detect_aruco(image, K, D, 0.029, True,
        aruco_dict=cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000))
    if arucos.n != 1:
        return

    # aruco
    pose = arucos.get_pose(0, 0)
    correction = np.eye(4)
    correction[2, 3] = -0.011
    pose = np.matmul(pose, correction)
    aruco_rotation = cv2.Rodrigues(pose[:3, :3])[0][:, 0]
    aruco_translation = pose[:3, 3]

    # pc
    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        pose = np.matmul(box_pose, correction)
    else:
        pose = box_pose
    pc_rotation = cv2.Rodrigues(pose[:3, :3])[0][:, 0]
    pc_translation = pose[:3, 3]

    rot_error = np.linalg.norm(aruco_rotation - pc_rotation) * 180 / np.pi
    trans_error = np.linalg.norm(aruco_translation - pc_translation) * 100

    rot_errors.append(rot_error)
    trans_errors.append(trans_error)

    correction = np.eye(4)
    correction[:3, :3], _ = cv2.Rodrigues(np.array([0, np.pi, 0]))
    correction[:3, 3] = np.array([0.11 / 2, -0.05 / 2, 0.022 / 2])

    # aruco
    pose = np.eye(4)
    pose[:3, :3], _ = cv2.Rodrigues(aruco_rotation)
    pose[:3, 3] = aruco_translation
    pose = np.matmul(pose, correction)
    rvec, _ = cv2.Rodrigues(pose[:3, :3])
    tvec = pose[:3, 3]
    cv2.drawFrameAxes(image, K, D, rvec, tvec, 0.10, 1)

    # pc
    pose = np.eye(4)
    pose[:3, :3], _ = cv2.Rodrigues(pc_rotation)
    pose[:3, 3] = pc_translation
    pose = np.matmul(pose, correction)
    rvec, _ = cv2.Rodrigues(pose[:3, :3])
    tvec = pose[:3, 3]
    cv2.drawFrameAxes(image, K, D, rvec, tvec, 0.07, 1)

camera.start()
stream(camera, callbacks=callback)
camera.stop()

print(sum(rot_errors) / len(rot_errors))
print(max(rot_errors))

print(sum(trans_errors) / len(trans_errors))
print(max(trans_errors))